In [26]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.agents import initialize_agent
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')

In [28]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

In [29]:

import pypdf

In [30]:
doc=PyPDFDirectoryLoader(path="data/")

In [31]:
data=doc.load()
len(data)

13

In [32]:
data[4]

Document(page_content="Home remedies for 'Bronchitis in children': \nMix 1 teaspoon oil of garlic (lasan) and 3 teaspoon honey and give a small amount three times a day to \nthe child. \n \nHome remedies for 'Burns by hot water': \nTake the thin buds of banana leaves. Bandage directly on affected areas. Tie the upper part for two days \nand then lower parts for two more days. burns, scar. Boil 1 cup neem bark (neem ki khal) in 4 cups water. \nRemove from fire and shake liquid. Apply the emerging froth on the affected area. Repeat several times \nand for several days. \n \nHome remedies for 'Body Burns': \nMash a ripe banana and apply on burns. Bandage with betel leaves (pan ka patta). If you have a minor \nburn, immediately place an ice pack on the burnt area for 10 minutes. Combine 4 teaspoon each of lime \njuice, coconut oil and rub until the betel leaves mixture turns white. Apply on affected parts. Apply curry \nleaves (kari patta) as poultices over affected areas. Spread a thin la

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)

In [35]:
data_chunks=text_splitter.split_documents(data)

In [36]:
database_dir="Database"

In [37]:
from langchain_community.vectorstores import Chroma

In [38]:
vector_db=Chroma.from_documents(
    documents=data_chunks,
    embedding=embeddings,
    persist_directory=database_dir
)

In [39]:
retriver=vector_db.as_retriever()

In [40]:
out=retriver.get_relevant_documents("home remedies for cough and cold")
print(out)

[Document(page_content="Home remedies for 'Cold/Cough': \n20 ml decoction prepared from 5 gm coarse powder sugar and turmeric powder thrice a day. or use of \ndhania powder as herbal tea every morning prevents cold, cough & problems related to digestion. \n \nHome remedies for 'Cold/Cough': \n20 ml decoction prepared from 5 gm Dhania, 1 gm Methi seeds, little turmeric powder should be taken 2-3\ntimes/day. Little elaichi powder with a tea spoon full of honey 3-4 times day. Even chewing elaichi (Not \nmore than 3 per day) frequently is help full in dry as w ell as productive cough. \n \n \nHome remedies for 'Cold/Cough': \nWarm decoction of 2 gm jeera, 5 gm Dhania, 1 gm Haldi, 1 gm Methi powder and little pepper should be \ntaken with honey/ sugar with lemon two to three times. Either decoction as mentioned above or chewing \nfew grains frequently helps in dry as well as productive cough.", metadata={'page': 0, 'source': 'data\\data.pdf'}), Document(page_content="Home remedies for 'Cold

In [41]:
from langchain.agents import Tool

rag_tool=Tool(
    name="Rag_tool",
    func=retriver.get_relevant_documents,
    description="this tool is recommended for getting home remedies for acute health problems"
)

In [42]:
search=DuckDuckGoSearchRun()

In [43]:
model = ChatGoogleGenerativeAI(model="gemini-pro")

In [44]:
agent=initialize_agent(
    llm=model,
    tools=[search,rag_tool],
    verbose=True,
    handle_parsing_errors=True
)

d:\DA\hack4bengal\MediConnect\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [45]:
out=agent.invoke("hello, suggest me some home remedie for headache")



> Entering new AgentExecutor chain...
Action: Rag_tool
Action Input: query: 'home remedies for headache'
Observation: [Document(page_content="Home remedies for 'Headache': \nRoast some ajwain dry on a tawa. Tie it in a muslin bag and sniff frequently. Make a paste of 2-3 \npowdered cloves (laung) and salt. Apply this paste on the forehead. A paste made of dry ginger (sonth) \nwith a little water or milk when applied to the forehead also gives relief. A ripe apple, after removing the \nupper rind and the inner hard portion should be taken with a little salt every morning on an empty \nstomach. Continue for a week. This yields good results even in case of chronic headaches. Mix 1 \nteaspoon finely ground cinnamon (dalchini) in 1 teaspoon water and apply on the forehead. It is very \neffective in headache due to exposure to cold air. Crush an onion and apply the paste on the head. Grind \n10-15 tulsi leaves with 4 cloves (laung) and 1 teaspoon dried ginger (sonth) into a paste and apply

In [46]:
out["output"]

'Roast some ajwain dry on a tawa and tie it in a muslin bag to sniff frequently.'

In [47]:
out=agent.invoke("hello, is it ok to eat papaya during pregnancy")
out["output"]



> Entering new AgentExecutor chain...
I should search duckduckgo for the answer
Action: duckduckgo_search
Action Input: is it ok to eat papaya during pregnancy
Observation: Papaya is generally safe to eat during pregnancy but only when it is fully ripe. Semi-ripe or green papaya should be avoided. It's important to know which fruits and vegetables are safe to eat ... Unripe papaya can raise vascular pressure and even lead to internal haemorrhage or bleeding in the placenta. Placental bleeding or haemorrhage can cause pregnancy and delivery-related complications. Papaya can affect the development and function of the placenta. 5. Increases Bowel Movement. Experts Weigh In. Pregnancy > Lifestyle. Is Papaya Safe During Pregnancy? Yes, If You Follow a Few Guidelines. Food & Nutrition. A couple of years after having my first baby, a fellow journalist reached out about a story she was working on, asking for some thoughts on what I would have told myself, advice-wise, about being pregnant. M

'Papaya is generally safe to eat during pregnancy but only when it is fully ripe. Semi-ripe or green papaya should be avoided.'